In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raultruco/cryptowatson-indicators#egg=cryptowatson-indicators
  %pip install --quiet git+https://github.com/mementum/backtrader.git@0fa63ef4a35dc53cc7320813f8b15480c8f85517#egg=backtrader

import backtrader as bt
from cryptowatsonindicators import datas, strategies
import requests
requests.packages.urllib3.disable_warnings()
# %matplotlib inline

# Rainbow Weighted Average Simulator

Usage...

### Variables

In [ ]:
strategy             = "rebalance"    # Select strategy between "weighted_dca" and "rebalance"
ticker_symbol        = "BTCUSDT"      # currently only works with BTCUSDT
start                = '01/08/2022'   # start date of the simulation. Ex: '01/08/2020' or None
end                  = None           # end date of the simulation. Ex: '01/08/2020' or None
initial_cash         = 10000.0        # initial broker cash. Default 10000 usd
min_order_period     = 5              # Minimum period in days to place orders
weighted_buy_amount  = 100            # Amount purchased in standard DCA
# weighted_multipliers = [0, 0.1, 0.2, 0.35, 0.5, 0.75, 1, 2.5, 3]      # Default order amount multipliers (weighted) for each index
weighted_multipliers = [0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.3, 2.1, 3.4]    # Fibonacci order amount multipliers (weighted) for each index
rebalance_percents   = [10, 20, 30, 40, 50, 60, 70, 80, 90]         # rebalance percentages for each index

### Run Simulator!

In [ ]:

cerebro = bt.Cerebro(stdstats=True)

# Data loader
data_loader = datas.DataLoader().load_data('ticker_nasdaq')

# Add the data feeds to Cerebro
ticker_data_feed = data_loader.to_backtrade_feed('ticker_nasdaq', start, end)
rwa_data_feed =  data_loader.to_backtrade_feed('ticker_nasdaq')
cerebro.adddata(ticker_data_feed)
cerebro.adddata(rwa_data_feed)

if strategy == "weighted_dca":
    cerebro.addstrategy(strategies.RwaWeightedAverageStrategy, weighted_buy_amount=weighted_buy_amount,
                    min_order_period=min_order_period, weighted_multipliers=weighted_multipliers, log=True)
elif strategy == "rebalance":
    cerebro.addstrategy(strategies.RwaRebalanceStrategy, min_order_period=min_order_period, rebalance_percents=rebalance_percents, log=True)
else:
    error_message = f"Invalid strategy: '{strategy}'"
    print(f"Error: {error_message}")
    exit()

# Add cash to the virtual broker
cerebro.broker.setcash(initial_cash)    # default: 10k

start_portfolio_value = cerebro.broker.getvalue()
cerebro.run()

end_portfolio_value = cerebro.broker.getvalue()     # Value in USDT
pnl_portfolio_value = end_portfolio_value - start_portfolio_value

position = cerebro.getbroker().getposition(data=ticker_data_feed)   # position: size: BTC in portfolio, price: average BTC purchase price
start_btc_price, end_btc_price = data_loader.get_value_start_end(label = 'ticker_nasdaq', start = start, end = end)

# btc price and PnL at start
start_pnl_value = pnl_portfolio_value + (position.size * start_btc_price)
start_pnl_percent = (start_pnl_value / start_portfolio_value) * 100
start_pnl_sign = '' if start_pnl_value < 0 else '+'

# btc price and PnL at the end
end_pnl_value = pnl_portfolio_value + (position.size * end_btc_price)
end_pnl_percent = (end_pnl_value / start_portfolio_value) * 100
end_pnl_sign = '' if end_pnl_value < 0 else '+'

# btc price and PnL at the end
avg_btc_price = position.price
avg_pnl_value = pnl_portfolio_value + (position.size * avg_btc_price)
avg_pnl_percent = (avg_pnl_value / start_portfolio_value) * 100
avg_pnl_sign = '' if avg_pnl_value < 0 else '+'

print("\nSIMULATION RESULT")
print("------------------------")
print(f"{'Started:':<12} {start_portfolio_value:<.2f} USD")
print(f"{'Ended:':<12} {end_portfolio_value:<.2f} USD, {position.size:6f} BTC")
print(f"{'PnL:':<12} {end_pnl_sign}{end_pnl_value:.2f} USD ({end_pnl_sign}{end_pnl_percent:.2f}%) at the latest price of {end_btc_price:.4f} USD")
print(f"{'PnL:':<12} {avg_pnl_sign}{avg_pnl_value:.2f} USD ({avg_pnl_sign}{avg_pnl_percent:.2f}%) on average price of {avg_btc_price:.4f} USD")
print(f"{'PnL:':<12} {start_pnl_sign}{start_pnl_value:.2f} USD ({start_pnl_sign}{start_pnl_percent:.2f}%) at the initial price of {start_btc_price:.4f} USD")

### Plot results

In [ ]:
enable_plot_results = False   # Set True to plot the simulation results

if enable_plot_results:
   cerebro.plot(volume=False)  # iplot=False, style='bar' , stdstats=False 